In [1]:
import pandas as pd
import json
import json
from tqdm import tqdm
import pandas as pd
import sys
sys.path.append("../src")
import prompt_utils
import os
import random

# vicuna 
# with rules classification only (0.76)
vicuna_base_path = "../data/vicuna_4bit/"
vicuna_with_rules_classification_only_name = "generic_prompt_with_rules_only_classification"
vicuna_with_rules_classification_only_func = prompt_utils.get_vicuna_prompt_with_rules_only_classification

# OA LLAMA
# Classification Only V03 (0.81)
# 1 pos 1 neg (0.79)
oa_base_path = "../data/openassistant_llama_30b_4bit/"
oa_classification_only_v03_name = "generic_prompt_without_context_only_classification_v03"
oa_classification_only_v03_func = prompt_utils.get_openassistant_llama_30b_4bit_without_context_only_classification_v03
oa_1pos_1neg_path_name = "generic_prompt_few_shot_prompt_only_classification_1_pos_1_neg_example"
oa_1pos_1neg_path_func = prompt_utils.get_openassistant_llama_30b_4bit_few_shot_prompt_only_classification_1_pos_1_neg_example

# Text Davinci
# Elaboration First V02 (0.94)
davinci_base_path = "../data/openai_text_davinci_003/"
davinci_elaboration_first_v02_name = "generic_prompt_without_context_elaboration_first_v02"
davinci_elaboration_first_v02_func = prompt_utils.get_openai_prompt_without_context_elaboration_first_v02

# Define a list of filenames to load
labeled_data_filename = "../data/labeled_data/generic_test_0.json"

dfs = []
with open(labeled_data_filename) as f:
    data = json.load(f)
df = pd.DataFrame(data["train"])
dfs.append(df)
df = pd.DataFrame(data["test"])
dfs.append(df)
df = pd.DataFrame(data["valid"])
dfs.append(df)
df_all = pd.concat(dfs)
all_labels = ["War/Terror", "Conspiracy Theory", "Education", "Election Campaign", "Environment", 
              "Government/Public", "Health", "Immigration/Integration", 
              "Justice/Crime", "Labor/Employment", 
              "Macroeconomics/Economic Regulation", "Media/Journalism", "Religion", "Science/Technology"]

unlabeled_dataset = prompt_utils.generate_unlabeled_dataset()

/home/bruno/Documents/fhnw/ip6-twitter-misinformation/notebooks/../src/prompt_utils.py:205: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/home/bruno/Documents/fhnw/ip6-twitter-misinformation/notebooks/../src/prompt_utils.py:205: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/home/bruno/Documents/fhnw/ip6-twitter-misinformation/notebooks/../src/prompt_utils.py:205: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [28]:
eng_tweets = unlabeled_dataset.loc[unlabeled_dataset.tweet_language == "en"].sample(frac=1, random_state = 42).reset_index()

In [29]:
eng_tweets.index == 1254904

array([False, False, False, ..., False, False, False])

In [10]:
start_index = 0
# start from last index (first row where no prediction was made)
# only label a couple of labels?

for i in tqdm(range(start_index, len(eng_tweets)), total = len(eng_tweets)):
    for label in all_labels:
        tweet_text = prompt_utils.normalize_tweet_simplified(eng_tweets.iloc[i]['tweet_text'])
        prompt, followup, request_params = prompt_utils.get_openassistant_llama_30b_4bit_without_context_only_classification_v03(tweet_text, label, prompt_utils.get_base_request_params())
        response = prompt_utils.get_response(request_params, prompt, "")
        print(prompt)
        print(response)
        new_column_name = f'{label}_pred'
        eng_tweets.loc[0, new_column_name] = response

  0%|          | 19/1459584 [00:00<4:06:42, 98.60it/s]

after the #US convoy in #Syria #DeirEzZor had been fired , the Americans raised two #helicopters , one of which destroyed a residential building with a rocket #USAoutOfSyria [url]
Assign 1 if the tweet is about War/Terror. Assign 0 if it is not about War/Terror.

Tweet: after the #US convoy in #Syria #DeirEzZor had been fired , the Americans raised two #helicopters , one of which destroyed a residential building with a rocket #USAoutOfSyria [url]
Class: 
after the #US convoy in #Syria #DeirEzZor had been fired , the Americans raised two #helicopters , one of which destroyed a residential building with a rocket #USAoutOfSyria [url]
Assign 1 if the tweet is about Conspiracy Theory. Assign 0 if it is not about Conspiracy Theory.

Tweet: after the #US convoy in #Syria #DeirEzZor had been fired , the Americans raised two #helicopters , one of which destroyed a residential building with a rocket #USAoutOfSyria [url]
Class: 
after the #US convoy in #Syria #DeirEzZor had been fired , the Ameri

  0%|          | 39/1459584 [00:00<4:29:29, 90.27it/s]

Militants from Jabhat al-Nusra have attempted to bombard the western parts of #Aleppo city , this time in the Hamdaniyah district . Four people were injured and two houses were destroyed . #Syria [url]
Assign 1 if the tweet is about Election Campaign. Assign 0 if it is not about Election Campaign.

Tweet: Militants from Jabhat al-Nusra have attempted to bombard the western parts of #Aleppo city , this time in the Hamdaniyah district . Four people were injured and two houses were destroyed . #Syria [url]
Class: 
Militants from Jabhat al-Nusra have attempted to bombard the western parts of #Aleppo city , this time in the Hamdaniyah district . Four people were injured and two houses were destroyed . #Syria [url]
Assign 1 if the tweet is about Environment. Assign 0 if it is not about Environment.

Tweet: Militants from Jabhat al-Nusra have attempted to bombard the western parts of #Aleppo city , this time in the Hamdaniyah district . Four people were injured and two houses were destroyed .

  0%|          | 49/1459584 [00:00<5:41:56, 71.14it/s]

...

  0%|          | 78/1459584 [00:00<5:12:46, 77.77it/s]

Stocks of staple food in the camp of #Er-Rukban end , said the official representative of the UN Office for the Coordination of Humanitarian Affairs Jens Lerke . He is worried about the situation in the camp and calls on the #US to open a #green corridor [url]
Assign 1 if the tweet is about Immigration/Integration. Assign 0 if it is not about Immigration/Integration.

Tweet: Stocks of staple food in the camp of #Er-Rukban end , said the official representative of the UN Office for the Coordination of Humanitarian Affairs Jens Lerke . He is worried about the situation in the camp and calls on the #US to open a #green corridor [url]
Class: 
Stocks of staple food in the camp of #Er-Rukban end , said the official representative of the UN Office for the Coordination of Humanitarian Affairs Jens Lerke . He is worried about the situation in the camp and calls on the #US to open a #green corridor [url]
Assign 1 if the tweet is about Justice/Crime. Assign 0 if it is not about Justice/Crime.

Tw

  0%|          | 108/1459584 [00:01<4:44:54, 85.38it/s]

" Despite the fact that our country has always been famous for its hospitality , but the actions of the United States and the international coalition that they are illegally invading our land force me to openly say that the West is not welcome in Syria ! " Professor Ali Mustafa [url]
Assign 1 if the tweet is about Religion. Assign 0 if it is not about Religion.

Tweet: " Despite the fact that our country has always been famous for its hospitality , but the actions of the United States and the international coalition that they are illegally invading our land force me to openly say that the West is not welcome in Syria ! " Professor Ali Mustafa [url]
Class: 
" Despite the fact that our country has always been famous for its hospitality , but the actions of the United States and the international coalition that they are illegally invading our land force me to openly say that the West is not welcome in Syria ! " Professor Ali Mustafa [url]
Assign 1 if the tweet is about Science/Technology.

KeyboardInterrupt: 

In [ ]:
for label in all_labels:
    prompt, followup, request_params = prompt_utils.get_openassistant_llama_30b_4bit_without_context_only_classification_v03("Hey how are you?", "War/Terror", prompt_utils.get_base_request_params())
#print(prompt)
        
#request_params['regenerate'] = False
#request_params['do_sample'] = False
request_params['max_new_tokens'] = 200
response = prompt_utils.get_response(request_params, prompt, "")
print(response)